# Foundations of Computer Science Project

## Import packages

In [1]:
import pandas as pd
import numpy as np
import re

## Import datasets

In [48]:
data_ps = pd.read_csv('googleplaystore.csv') #Import the datasets
data_ur = pd.read_csv('googleplaystore_user_reviews.csv')

In [3]:
data_ps.head() #Check the googleplaystore dataset

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [41]:
data_ur.head() #Check the googleplaystore_user_reviews dataset

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [49]:
#During the exercise 2 i found that the record 10472 has inaccurate values so i decided to remove it.
data_ps = data_ps.drop(10472)

## 1. Convert the app sizes to a number

In [50]:
data_ps['Size'] = [re.sub('Varies with device', '99999999', size) for size in data_ps['Size']] #Assign to 'Varies with device' a new numeric value 
data_ps['Size'] = [re.sub(',', '.', size) for size in data_ps['Size']] #Format 

In [44]:
sizes_to_num_re = re.compile('(?P<amount>\d*\.*\d*)(?P<unit>\w*\+*)') #

def unit_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000 #Identifying the number of bytes into 1 Gb, Mb and Kb
    if unit == 'k':
        return 1000
    return 1

def to_numeric(elem): #Converting the strings into numbers
    m = sizes_to_num_re.search(elem)
    unit = m.group('unit')
    mult = unit_to_mult(unit)
    amount = float(m.group('amount'))
    return int(amount * mult)

In [51]:
data_ps['ExtSize'] = data_ps['Size'].apply(to_numeric) #Applying the function

In [52]:
data_ps['Size'] = [re.sub('99999999', 'Varies with device', size) for size in data_ps['Size']] #Reconverting the value '99999999' into 'Varies with device'
data_ps['ExtSize'] = data_ps['ExtSize'].apply(str)
data_ps['ExtSize'] = [re.sub('99999999', 'Varies with device', size) for size in data_ps['ExtSize']]
data_ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000


## 2. Convert the number of installs to a number

In [53]:
data_ps.dtypes #Check the dataframe's columns dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize            object
dtype: object

In [54]:
data_ps.groupby('Installs')['Installs'].size() #Check for some strange values

Installs
0                    1
0+                  14
1+                  67
1,000+             907
1,000,000+        1579
1,000,000,000+      58
10+                386
10,000+           1054
10,000,000+       1252
100+               719
100,000+          1169
100,000,000+       409
5+                  82
5,000+             477
5,000,000+         752
50+                205
50,000+            479
50,000,000+        289
500+               330
500,000+           539
500,000,000+        72
Name: Installs, dtype: int64

In [55]:
data_ps['NumInstalls'] = [re.sub(',', '', number) for number in data_ps['Installs']] #Remove the comma

In [68]:
data_ps['NumInstalls'] = [re.sub('\+*', '', number) for number in data_ps['NumInstalls']] #Remove the plus
data_ps['NumInstalls'] = data_ps['NumInstalls'].apply(float) #Converting to float

In [58]:
data_ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000,10000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000,500000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000,5000000
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000,50000000
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000,100000


## 3. Transform “Varies with device” into a missing value

In [78]:
data_psnan = data_ps.replace('Varies with device', np.nan)
data_psnan['ExtSize'] = data_ps['ExtSize'].apply(float) #Converting into float

In [79]:
data_psnan.dtypes #Checking

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize           float64
NumInstalls       float64
dtype: object

## 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [80]:
data_psnan.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2 and up,25000000.0,50000000.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0


In [81]:
data_psnan.isnull().any() #Checking for missing values

App               False
Category          False
Rating             True
Reviews           False
Size               True
Installs          False
Type               True
Price             False
Content Rating    False
Genres            False
Last Updated      False
Current Ver        True
Android Ver        True
ExtSize            True
NumInstalls       False
dtype: bool

In [134]:
data_nomis = data_psnan.dropna()
data_nomis.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5600000.0,50000.0


In [135]:
data_nomis.groupby('Current Ver').size().head()

Current Ver
0.0.0.2    1
0.0.1      6
0.0.2      3
0.0.3      1
0.0.4      2
dtype: int64

In [129]:
pd.options.mode.chained_assignment = None

In [136]:
data_nomis['CurrVer'] = [re.sub('\\D', '', row) for row in data_nomis['Current Ver']] #Remove everything that is not a number from Current Ver

In [137]:
data_nomis['CurrVer'] = [re.sub(r'(?<!^)(?=(\d{1})+$)', r'.', numb) for numb in data_nomis['CurrVer']]

In [138]:
data_nomis.head() #Do it

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0,1.0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0,1.2.4
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0,1.1
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5600000.0,50000.0,1.0


In [123]:
#Now on Android Ver

7637